In [2]:
# 8th July, 2019
#Author Esha Sarkar
import pandas
import numpy as np
import keras

from random import seed
from random import randint
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from sklearn.metrics import mean_squared_error
from keras import optimizers
from random import seed
from random import randint
from numpy import array
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import LSTM,Dropout,Reshape
from math import sqrt
from sklearn.metrics import mean_squared_error
import tensorflow as tf

Using TensorFlow backend.


In [3]:
TagGenotypes_10k = np.array(pandas.read_csv('sorted_tag_SNPs_10k_genotypes.txt', sep='\s+', delimiter=None, header=None, names=None, index_col=None))
TargetGenotypes = np.array(pandas.read_csv('sorted_target_SNP_genotypes.txt', sep='\s+', delimiter=None, header=None, names=None, index_col=None))

In [4]:
DatasetX1_10k = TagGenotypes_10k[:,4:].transpose()
DatasetY1_10k = TargetGenotypes[:,4:].transpose()
ShufflingVector_10k = np.random.permutation(DatasetX1_10k.shape[0])
DatasetX_10k = DatasetX1_10k[ShufflingVector_10k]
DatasetY_10k = DatasetY1_10k[ShufflingVector_10k]

In [5]:
def OneHotEncoder(Matrix):
    Temp = np.zeros((Matrix.shape[0],Matrix.shape[1],3))
    for num, i in enumerate(Matrix):
        Temp[num] = (tf.keras.utils.to_categorical(i,3))
    return (Temp)

In [6]:
split = int(0.8*(DatasetX_10k.shape[0]))
Xtr1_10k = DatasetX_10k[:split]
Xtr_10k = (OneHotEncoder(Xtr1_10k)).reshape((Xtr1_10k.shape[0],Xtr1_10k.shape[1]*3))

Xts1_10k = DatasetX_10k[split:]
Xts_10k = (OneHotEncoder(Xts1_10k)).reshape((Xts1_10k.shape[0],Xts1_10k.shape[1]*3))

ytr1_10k = DatasetY_10k[:split]
ytr_10k = (OneHotEncoder(ytr1_10k)).reshape((ytr1_10k.shape[0],ytr1_10k.shape[1]*3))

yts1_10k = DatasetY_10k[split:]
yts_10k = (OneHotEncoder(yts1_10k)).reshape((yts1_10k.shape[0],yts1_10k.shape[1]*3))

In [7]:
np.savetxt('XTrain_10k.txt', Xtr1_10k, delimiter=',',fmt='%1d') 
np.savetxt('XTest_10k.txt', Xts1_10k, delimiter=',',fmt='%1d') 
np.savetxt('YTrain_10k.txt', ytr1_10k, delimiter=',',fmt='%1d') 
np.savetxt('YTest_10k.txt', yts1_10k, delimiter=',',fmt='%1d') 

In [8]:
def Accuracy(X,y,ModelName):
    y_actual = np.argmax(y.reshape((y.shape[0],int(y.shape[1]/3),3)),2)
    y1 = (ModelName.predict(X))
    y_hat = np.argmax((y1.reshape((y1.shape[0],int(y1.shape[1]/3),3))),2)
    A = (np.count_nonzero(y_hat==y_actual))/(y_actual.shape[0]*y_actual.shape[1])
    return (A)

In [9]:
TagNos = DatasetX_10k.shape[1]
from keras.constraints import maxnorm, nonneg
from keras import backend as K
K.clear_session()
model_10k = Sequential()
model_10k.add(Dense(1500, input_dim=TagNos*3,bias_constraint=keras.constraints.NonNeg(),kernel_constraint=keras.constraints.NonNeg()))
opt = keras.optimizers.Adam(lr=0.0008, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model_10k.compile(loss='mean_squared_error', optimizer=opt)
model_10k.summary()  

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1500)              4704000   
Total params: 4,704,000
Trainable params: 4,704,000
Non-trainable params: 0
_________________________________________________________________


In [10]:
kFolds = 5
kFoldAccuracy = []
split_value = int((1/kFolds)*Xtr_10k.shape[0])
for fold in range(kFolds):
    Testfrom = fold*split_value
    Testto = (fold+1)*split_value
    
    Trainto1 = (fold)*split_value    
    Trainfrom2 = (fold+1)*split_value
    print ("========================================================================================================")
    print ("KFOLD: ",fold)
    print ("Test:", Testfrom,Testto)
    print ("Train: [",0,Trainto1,"] [",Testto,Xtr_10k.shape[0],"]")
    
    X_train = ((np.concatenate((Xtr_10k[:Trainto1],Xtr_10k[Trainfrom2:])))).astype('float')
    X_ts = (Xtr_10k[Testfrom:Testto]).astype('float')

    y_train = ((np.concatenate((ytr_10k[:Trainto1],ytr_10k[Trainfrom2:])))/2).astype('float')
    y_ts = (ytr_10k[Testfrom:Testto]).astype('float')
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5,
                              patience=3, min_lr=0.00001)
    model_10k.fit(X_train, y_train, epochs=30, verbose=1,callbacks=[reduce_lr])
    a=Accuracy(X_train,y_train,model_10k)
    aprime=Accuracy(X_ts,y_ts,model_10k)
    print ("Training accuracy: ", np.mean(a))
    print ("Test: ", np.mean(aprime))
    kFoldAccuracy.append(np.mean(aprime))
    print ("========================================================================================================")

KFOLD:  0
Test: 0 400
Train: [ 0 0 ] [ 400 2003 ]
Instructions for updating:
Use tf.cast instead.
Epoch 1/30
1603/1603 [==============================] - 10s 6ms/step - loss: 18.8468
Epoch 2/30
1603/1603 [==============================] - 8s 5ms/step - loss: 0.1140
Epoch 3/30
1603/1603 [==============================] - 8s 5ms/step - loss: 0.0455
Epoch 4/30
1603/1603 [==============================] - 9s 5ms/step - loss: 0.0410
Epoch 5/30
1603/1603 [==============================] - 9s 5ms/step - loss: 0.0385
Epoch 6/30
1603/1603 [==============================] - 9s 5ms/step - loss: 0.0364
Epoch 7/30
1603/1603 [==============================] - 9s 6ms/step - loss: 0.0346
Epoch 8/30
1603/1603 [==============================] - 8s 5ms/step - loss: 0.0330
Epoch 9/30
1603/1603 [==============================] - 8s 5ms/step - loss: 0.0315
Epoch 10/30
1603/1603 [==============================] - 8s 5ms/step - loss: 0.0302
Epoch 11/30
1603/1603 [==============================] - 8s 5ms/step 

In [11]:
Accuracy(Xts_10k,yts_10k,model_10k)

0.850934131736527

In [12]:
Accuracy(Xtr_10k,ytr_10k,model_10k)

0.882191712431353

In [13]:
Layers_10k = model_10k.layers
W_10k,b_10k = Layers_10k[0].get_weights()
W_10k[W_10k==-0.0]=0.0
b_10k[b_10k==-0.0]=0.0

In [17]:
W_10k = (W_10k*10000).astype('int')
b_10k = (b_10k*10000).astype('int')

In [30]:
zeros = ((W_10k.shape[0]*W_10k.shape[1])-(np.count_nonzero(W_10k)))/(W_10k.shape[0]*W_10k.shape[1])

In [18]:
np.count_nonzero(W_10k)/(3135*1500)

0.23891270600744285

In [17]:
W_10k.shape

(3135, 1500)

In [15]:
np.savetxt('W_10k_complete.txt', W_10k, delimiter=',',fmt='%1.4f') 

In [ ]:
Layers_10k = model_10k.layers
W_10k,b_10k = Layers_10k[0].get_weights()
W_10k[W_10k==-0.0]=0.0
b_10k[b_10k==-0.0]=0.0

In [ ]:
b[b==-0.0]=0.0

In [ ]:
np.savetxt('W_10k_complete.txt', W_10k, delimiter=',',fmt='%1.4f') 
np.savetxt('b_10k_complete.txt', b_10k, delimiter=',',fmt='%1.4f') 

In [ ]:
y1 = model_10k.predict(Xts_10k)
yts_10k_hat = np.argmax((y1.reshape((y1.shape[0],int(y1.shape[1]/3),3))),2)

In [ ]:
O = np.dot(Xts_10k,W)+b
O_num = np.argmax((O.reshape((O.shape[0],int(O.shape[1]/3),3))),2)

In [ ]:
for u in O_num==yts_10k_hat:
    print (True in u)

In [ ]:
def scale(Matrix):
    min_value = np.min(Matrix)
    max_value = np.max(Matrix)
    ScaledMatrix = (Matrix-min_value)/(max_value-min_value)
    return (ScaledMatrix)

In [ ]:
yts_hat_normalized = scale(model.predict(Xts))

In [ ]:
temp = yts_hat_normalized.reshape((500,500,3))

In [ ]:
divs = np.array([np.sum(i,1) for i in temp])

In [ ]:
yts_preds = np.zeros((temp.shape))
for i in range(500):
    for j in range(500):
        yts_preds[i][j] = temp[i][j]/divs[i][j]

In [ ]:
temp2 = yts_preds.reshape((250000,3))

In [ ]:
temp2

In [ ]:
yts_print = yts.reshape((500,500,3)) 

In [ ]:
yts.shape

In [ ]:
np.savetxt('y_pred.txt', temp2, delimiter=',',fmt='%1.6f') 

In [ ]:
np.savetxt('y_actual.txt', yts1.astype('int').reshape((250000,1)), delimiter=',',fmt='%1d') 

## Homomorphic part

In [ ]:
W = np.loadtxt('W.txt',delimiter=',')
b = np.loadtxt('b.txt',delimiter=',')

In [ ]:
import time
start_time = time.time()
np.dot(Xts,W)+b
print ("Time required in plain-text",time.time()-start_time)

In [ ]:
from phe import paillier
puk, prk = paillier.generate_paillier_keypair(n_length=1024)

In [ ]:
Xts_10k.shape

In [ ]:
start_time = time.time()
Xts_e = EncryptVector(puk,Xts)
print (time.time()-start_time)

In [ ]:
xtemp1 = Xts.reshape((500*3135,1))

In [ ]:
pool.apply([puk.encrypt(i) for i in range(5)])

In [ ]:
import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())
start_time = time.time()
Xts_e1 = pool.apply(EncryptVector(puk,a))
print (time.time()-start_time)

In [ ]:
def EncryptVector(pubkey, x):
    xtemp = x.reshape((x.shape[0]*x.shape[1],1))
    return (np.array([pubkey.encrypt(xtemp[i][0]) for i in range(xtemp.shape[0])])).reshape(x.shape)

def DecryptVector(privkey, x):
    xtemp = x.reshape((x.shape[0]*x.shape[1],1))
    return (np.array([privkey.decrypt(i[0]) for i in xtemp])).reshape(x.shape)

def AddEncryptedVectors(x, y):

    if len(x) != len(y):
        raise Exception('Encrypted vectors must have the same size')

    return [x[i] + y[i] for i in range(len(x))]

In [ ]:
start_time = time.time()
Xts_e = EncryptVector(puk,Xts)
print (time.time()-start_time)

In [ ]:
start_time = time.time()
d = b+c_e
print (time.time()-start_time)

In [ ]:
Check_Xts = DecryptVector(prk,Xts_e)

In [ ]:
W.shape

In [ ]:
t1 = np.ones((10,3135))
t2 = np.ones((3135,1500))
t1_e = EncryptVector(puk,t1)
#t2_e = EncryptVector(puk,t2)

In [ ]:
start_time = time.time()
m_e = np.dot(t1_e,t2)
print (time.time()-start_time)

In [ ]:
start_time = time.time()
m_e = np.dot(t1_e,t2)
print (time.time()-start_time)

In [ ]:
m = DecryptVector(prk,m_e)

In [ ]:
a = (np.ones((500,1500)))*1.32
b = (np.ones((500,1500)))*2.32
start_time = time.time()
Xts_e = EncryptVector(puk,Xts)
print (time.time()-start_time)
#b_e = EncryptVector(puk,b)
#c_e = a_e + b
#c = DecryptVector(c_e)

In [ ]:
start_time = time.time()
Xts_e.dot(W)
print (time.time()-start_time)

In [ ]:
g = DecryptVector(prk,f)

In [ ]:
g

In [ ]:



def decrypt_vector(privkey, x):
    return np.array([privkey.decrypt(i) for i in x])


def sum_encrypted_vectors(x, y):

    if len(x) != len(y):
        raise Exception('Encrypted vectors must have the same size')

    return [x[i] + y[i] for i in range(len(x))]

In [ ]:
Xts.shape

In [ ]:
EncXts = []
for i in range(Xts.shape[0]):
    for j in range(Xts.shape[1]):
        #print (i,j)
        EncXts[i][j]=(puk.encrypt(Xts[i][j])).ciphertext()

In [ ]:
EncXts2 = []
for i in range(Xts.shape[0]):
    for j in range(Xts.shape[1]):
        #print (i,j)
        EncXts2.append(puk.encrypt(Xts[i][j]))

In [ ]:
EncXts2 = np.array(EncXts2)

In [ ]:
EncXts2 = EncXts2.reshape((Xts.shape))

In [ ]:
a = np.array([[9,5],[2,5],[1,7]])

In [ ]:
b = np.array([[puk.encrypt(6)],[puk.encrypt(11)]])
d = np.array([[6],[11]])

In [ ]:
f = np.array([[puk.encrypt(8)],[puk.encrypt(1)]])
g = np.array([[8],[1]])

In [ ]:
def MatMul(A,B):
    ResultsTemp = []
    for i in range(A.shape[0]):
        #print (i)
        for j in range(B.shape[1]):
            temp = 0
            for k in range(B.shape[0]):
                temp += A[i][k] * B[k][j]
            ResultsTemp.append(temp)
    ResultsTemp = np.array(ResultsTemp)
    Results = ResultsTemp.reshape((A.shape[0],B.shape[1]))
    return (Results)

In [ ]:
z = b+f

In [ ]:
W.shape

In [ ]:
import time
start_time = time.time()
e = MatMul(a,b)
print ("Time required to multiply",time.time()-start_time)

In [ ]:
for i in z:
    print (prk.decrypt(i[0]))

In [ ]:
e

In [ ]:
O = np.argmax((np.matmul(Xts,W)+b).reshape((504,500,3)),2)

In [ ]:
O2 = np.argmax((model.predict(Xts)).reshape((504,500,3)),2)

In [ ]:
np.count_nonzero((O!=O2))

In [ ]:
np.savetxt('W_10k.txt', W, delimiter=',') 

In [ ]:
np.savetxt('b_10k.txt',b,delimiter=',')

In [ ]:
def scale(Matrix):
    min_value = np.min(Matrix)
    max_value = np.max(Matrix)
    ScaledMatrix = (Matrix-min_value)/(max_value-min_value)
    return (ScaledMatrix)